In [19]:
# 离散化
from typing import List
from itertools import pairwise


class Node:

    def __init__(self, si, ei) -> None:
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.d = ei - si + 1
        self.left = None
        self.right = None
        self.val = 0  # 区间和
        self.lazy = 0  # 每个值+lazy 未下发


class SegmentTree:

    def __init__(self, si, ei) -> None:
        self.root = Node(si, ei)

    def push_down(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if not node.lazy: return
        node.left.val += node.left.d * node.lazy
        node.right.val += node.right.d * node.lazy
        node.left.lazy += node.lazy
        node.right.lazy += node.lazy
        node.lazy = 0

    def update(self, node: Node, l, r, val):  # [l,r]+val
        if node.ei < l or node.si > r:
            return
        if l <= node.si and node.ei <= r:
            node.val += node.d * val
            node.lazy += val
            return
        self.push_down(node)
        if node.mi >= l:
            self.update(node.left, l, r, val)
        if node.mi + 1 <= r:
            self.update(node.right, l, r, val)
        node.val = node.left.val + node.right.val

    def query(self, node: Node, l, r):  # [l,r]和
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si and node.ei <= r:
            return node.val
        self.push_down(node)
        ans = 0
        if node.mi >= l:
            ans += self.query(node.left, l, r)
        if node.mi + 1 <= r:
            ans += self.query(node.right, l, r)
        return ans


class Solution:

    def minAvailableDuration(self, slots1: List[List[int]], slots2: List[List[int]], duration: int) -> List[int]:
        lst = set()
        start = set()
        for x, y in slots1 + slots2:
            start.add(x)
            lst.add(x)
            lst.add(y)
        start = sorted(start)
        lst = sorted(lst)
        n = len(lst)
        ha = {c: i for i, c in enumerate(lst)}
        st = SegmentTree(0, n - 1)
        for x, y in slots1 + slots2:
            st.update(st.root, ha[x], ha[y], 1)
        for x in start:
            i = ha[x]
            if st.query(st.root, i, i + 1) == 4 and lst[i + 1] - lst[i] >= duration:  # 4: (i+1-i+1)*2
                return [lst[i], lst[i] + duration]
        return []


s = Solution()
s.minAvailableDuration([[10, 50], [60, 120], [140, 210]], [[0, 15], [40, 50]], 8)


[40, 48]

In [16]:
# 超时
from math import inf
from typing import List
from itertools import pairwise


class Node:

    def __init__(self, si, ei) -> None:
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.left = None
        self.right = None
        self.mx = 0  # 区间最大值
        self.mn = 0  # 区间最小值
        self.lazy = 0  # 每个值+lazy 未下发


class SegmentTree:

    def __init__(self, si, ei) -> None:
        self.root = Node(si, ei)

    def push_down(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if not node.lazy: return
        node.left.mx += node.lazy
        node.left.mn += node.lazy
        node.right.mx += node.lazy
        node.right.mn += node.lazy
        node.left.lazy += node.lazy
        node.right.lazy += node.lazy
        node.lazy = 0

    def update(self, node: Node, l, r, val):  # [l,r]+val
        if node.ei < l or node.si > r:
            return
        if l <= node.si and node.ei <= r:
            node.mx += val
            node.mn += val
            node.lazy += val
            return
        self.push_down(node)
        if node.mi >= l:
            self.update(node.left, l, r, val)
        if node.mi + 1 <= r:
            self.update(node.right, l, r, val)
        node.mx = max(node.left.mx, node.right.mx)
        node.mn = min(node.left.mn, node.right.mn)

    def query(self, node: Node, l, r):  # [l,r]最大值
        if node.ei < l or node.si > r:
            return 0, inf
        if l <= node.si and node.ei <= r:
            return node.mx, node.mn
        self.push_down(node)
        mx, mn = 0, inf
        if node.mi >= l:
            l1, l2 = self.query(node.left, l, r)
            mx = max(mx, l1)
            mn = min(mn, l2)
        if node.mi + 1 <= r:
            r1, r2 = self.query(node.right, l, r)
            mx = max(mx, r1)
            mn = min(mn, r2)
        return mx, mn


class Solution:

    def minAvailableDuration(self, slots1: List[List[int]], slots2: List[List[int]], duration: int) -> List[int]:
        lst = set()
        for x, y in slots1 + slots2:
            lst.add(x)
            lst.add(y)
        lst = sorted(lst)
        st = SegmentTree(lst[0], lst[-1])
        for x, y in slots1 + slots2:
            st.update(st.root, x, y, 1)
        for x, y in pairwise(lst):
            mx, mn = st.query(st.root, x, y)
            if mx == mn == 2 and y - x >= duration:
                return [x, x + duration]
        return []


s = Solution()
s.minAvailableDuration([[10, 50], [60, 120], [140, 210]], [[0, 15], [40, 50]], 8)


[40, 48]

In [17]:
# 超时
from math import inf
from typing import List
from itertools import pairwise


class Node:

    def __init__(self, si, ei) -> None:
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.d = ei - si + 1
        self.left = None
        self.right = None
        self.val = 0  # 区间和
        self.lazy = 0  # 每个值+lazy 未下发


class SegmentTree:

    def __init__(self, si, ei) -> None:
        self.root = Node(si, ei)

    def push_down(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if not node.lazy: return
        node.left.val += node.left.d * node.lazy
        node.right.val += node.right.d * node.lazy
        node.left.lazy += node.lazy
        node.right.lazy += node.lazy
        node.lazy = 0

    def update(self, node: Node, l, r, val):  # [l,r]+val
        if node.ei < l or node.si > r:
            return
        if l <= node.si and node.ei <= r:
            node.val += node.d * val
            node.lazy += val
            return
        self.push_down(node)
        if node.mi >= l:
            self.update(node.left, l, r, val)
        if node.mi + 1 <= r:
            self.update(node.right, l, r, val)
        node.val = node.left.val + node.right.val

    def query(self, node: Node, l, r):  # [l,r]和
        if node.ei < l or node.si > r:
            return 0
        if l <= node.si and node.ei <= r:
            return node.val
        self.push_down(node)
        ans = 0
        if node.mi >= l:
            ans += self.query(node.left, l, r)
        if node.mi + 1 <= r:
            ans += self.query(node.right, l, r)
        return ans


class Solution:

    def minAvailableDuration(self, slots1: List[List[int]], slots2: List[List[int]], duration: int) -> List[int]:
        lst = set()
        for x, y in slots1 + slots2:
            lst.add(x)
            lst.add(y)
        lst = sorted(lst)
        st = SegmentTree(lst[0], lst[-1])
        for x, y in slots1 + slots2:
            st.update(st.root, x, y, 1)
        for x, y in pairwise(lst):
            if st.query(st.root, x, y) == (y - x + 1) * 2 and y - x >= duration:
                return [x, x + duration]
        return []


s = Solution()
s.minAvailableDuration([[10, 50], [60, 120], [140, 210]], [[0, 15], [40, 50]], 8)


[40, 48]